In [ ]:
%pylab notebook
import numpy as np
import pandas as pd

In [ ]:
!ls data

In [ ]:
ytuna=pd.read_excel('data/YTuna_clean_graphs.xlsm',na_values='.')
waituku=pd.read_csv('data/Waituku.txt',delimiter='\t')

In [ ]:
print(ytuna['Flow 1'].dtype)
ytuna.columns

In [ ]:
#figure()
data=np.array(ytuna['TN/Cl'],dtype('float'))
np.nanstd(data)
data=data[~np.isnan(data)]
data.mean()
#data=data[~np.isnan(data)]
#hist(data,bins=50)

In [ ]:
figure()
from scipy.stats.mstats import zscore
intersting_cols=ytuna.columns[6:]
intersting_cols_next=[]
intersting_next=[]
def nanzscore(data):
    if np.nanstd(data) <1e-8:
        return None
    return (data-np.nanmean(data))/np.nanstd(data)

for ind,col in enumerate(intersting_cols):
    subplot(10,10,ind+1)
    #title(str(ind))
    data=np.array(ytuna[col])
    try:
        data=nanzscore(data)
        if data is None:
            print('skeeping..',ind,col)
            continue
        #data=zscore(data)
        hist(data[~np.isnan(data)])
        intersting_next.append(data)
        intersting_cols_next.append(col)
    except:
        print(ind,col)


In [ ]:
dim=len(intersting_next)
im=np.zeros((dim,dim))
for i in range(dim):
    for j in range(dim):
        if i==j:
            continue
        d1=intersting_next[i]
        d2=intersting_next[j]
        notnans=~(np.isnan(d1) | np.isnan(d2))
        if np.count_nonzero(notnans)<20:
            continue
        d1n=d1[notnans]
        d2n=d2[notnans]
        #im[i,j]=np.cov(d1[notnans],d2[notnans])/(np.nanstd(d1)*np.nanstd(d2)))[0,1]
        if d1n.std()<1e-5 or d2n.std()<1e-5:
            continue
        im[i,j]=np.abs(np.corrcoef(d1n,d2n)[1,0])
        
figure()
imshow(im)

In [ ]:
def get_pair(i,j):
    d1=intersting_next[i]
    d2=intersting_next[j]
    notnans=~(np.isnan(d1) | np.isnan(d2))
    return d1[notnans],d2[notnans],intersting_cols_next[i],intersting_cols_next[j]

i=15;j=6
x,y,xl,yl=get_pair(i,j)
figure()
plot(x,y,'+',alpha=0.7)
xlabel(xl)
ylabel(yl)
title('pearson coef {:.2f}'.format(im[i,j]))



In [ ]:
#ytuna['Carbonate']
import rpy2.robjects as ro

In [ ]:
#http://igraph.org/r/
ro.r('library(igraph)')

 

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]
tresh=0.8
edges=flatten([(intersting_cols_next[i]+'('+str(i)+')',
                intersting_cols_next[j]+'('+str(j)+')') for i in range(len(im)) for j in range(i) if im[i,j]>tresh])
weight=[im[i,j] for i in range(len(im)) for j in range(i) if im[i,j]>tresh]
#edges=list(map(str,edges))
#print(str(weight))
s='''
edges <- c({0})
g<-graph(edges, n=max(edges), directed=FALSE)
E(g)$weight <- c({1})
E(g)[ weight <= 0.9 ]$color <- "green"
E(g)[ weight > 0.9 ]$color <- "red"
E(g)[ weight <= 0.8 ]$width <- 3
E(g)[ weight > 0.8 ]$width <- 5
E(g)[ weight > 0.9 ]$width <- 7
E(g)$asp <- 1
#png('rplot.png', width = 2000, height = 2000)
jpeg('rplot.jpg', width = 2000, height = 2000)
#plot(g, layout = layout.fruchterman.reingold,vertex.label=V(g)$number)
plot(g, layout = layout.fruchterman.reingold, vertex.size=8, sub='hh')
#tkplot(g, canvas.width = 800, canvas.height =600)
dev.off()
'''.format(str(edges)[1:-1],str(weight)[1:-1])
G=ro.r(s)
figure()
imshow(imread('rplot.jpg'))

In [ ]:
del(G)

In [ ]:
#geo
figure()
plot(ytuna['Easting']/1000,ytuna['Northing']/1000,'*',alpha=0.3)
axis('equal')

In [ ]:

import plotly.plotly as py
mapbox_access_token = 'pk.eyJ1Ijoib3JpZzc0IiwiYSI6ImNqMXBzZGl4NjAwNW0zMm92bzJteXM5dTQifQ.TFP8iP0l1YDjX5BaLHWwkQ'
py.sign_in('orig74','SKJLoURDiuVeBwOO7RWY')
from plotly.graph_objs import *
#py.sign_in('username', 'api_key')
#
geos=[utm.to_latlon(e, n, 57,'G',strict=False) for e,n in zip(ytuna['Easting'],ytuna['Northing'])]
#not working utm look an next cell

data = Data([
    Scattermapbox(
        lat=[lat],
        lon=[lon],
        mode='markers',
        marker=Marker(
            size=14
        ),
        text=['Dun'],
    )
for lat,lon in geos])

layout = Layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        style='mapbox://styles/mapbox/satellite-streets-v9',
        bearing=0,
        center=dict(
            lat=-45,
            lon=170
        ),
        pitch=0,
        zoom=5
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='New Zealand Mapbox')

In [ ]:
#for e,n in np.vstack((ytuna['Easting'],ytuna['Northing'])).T:
#    print(e,n)

In [ ]:
##output from
##http://apps.linz.govt.nz/coordinate-conversion/index.aspx?Test=
#options from utm zone 59 to geo84
#Easting	Northing	       	Longitude	Latitude
geos='''1267164.000	4840209.000	       	-155.06202697	-46.15767353
       	1268105.000	4839101.000	       	-155.04823445	-46.16644055
       	1268105.000	4839101.000	       	-155.04823445	-46.16644055
       	1268105.000	4839101.000	       	-155.04823445	-46.16644055
       	1268105.000	4839101.000	       	-155.04823445	-46.16644055
       	1267026.000	4837117.000	       	-155.05879986	-46.18523809
       	1267026.000	4837117.000	       	-155.05879986	-46.18523809
'''
geos=[list(map(float,l.split('\t')[-2:]))[::-1] for l in geos.split('\n') if l.strip()]